In [178]:
import awkward as ak
import numpy as np
import uproot as uproot
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.5.0


In [179]:
file = uproot.open('/eos/cms/store/group/dpg_hgcal/comm_hgcal/hackathon/samples/close_by_double_pion/production/new_new_ntuples/ntuples_3933206_0.root')

In [180]:
file.keys()

['ntuplizer;1',
 'ntuplizer/simtrackstersSC;17',
 'ntuplizer/simtrackstersSC;16',
 'ntuplizer/simtrackstersCP;17',
 'ntuplizer/simtrackstersCP;16',
 'ntuplizer/tracksters;6',
 'ntuplizer/tracksters;5',
 'ntuplizer/clusters;4',
 'ntuplizer/clusters;3',
 'ntuplizer/graph;1',
 'ntuplizer/candidates;1',
 'ntuplizer/trackstersMerged;1',
 'ntuplizer/associations;1',
 'ntuplizer/tracks;1']

In [181]:
trackstersclue3d = file["ntuplizer/tracksters"]
simtrackstersCP = file["ntuplizer/simtrackstersCP"]
graph = file["ntuplizer/graph"]
associations = file["ntuplizer/associations"]
candidates = file["ntuplizer/candidates"]
tsmerged = file["ntuplizer/trackstersMerged;1"]
tsmerged.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
NTrackstersMerged    | unknown                  | <UnknownInterpretation 'non...
barycenter_x         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
barycenter_y         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
barycenter_z         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
EV1                  | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
EV2                  | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
EV3                  | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
eVector0_x           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
eVector0_y           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
eVector0_z           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
sigmaPCA1            | std::

In [182]:
# Get ntuples from tree in file
tracksterN = trackstersclue3d["NTracksters"].array(library='np')
x = trackstersclue3d["barycenter_x"].array(library='np')
y = trackstersclue3d["barycenter_y"].array(library='np')
z = trackstersclue3d["barycenter_z"].array(library='np')
rawE = trackstersclue3d["raw_energy"].array(library='np')
rawE_EM = trackstersclue3d["raw_em_energy"].array(library='np')
linkedInners = graph["linked_inners"].array(library='np')
linkedOuters = graph["linked_outers"].array(library='np')
simTracksters_CP_N = simtrackstersCP["stsCP_NTracksters"].array(library='np')
tsAssocMap = associations["tsCLUE3D_recoToSim_CP"].array(library='np')
tsAssocQual = associations["tsCLUE3D_recoToSim_CP_score"].array(library='np')

event = 10

In [183]:
import glob

input_folder = "/eos/cms/store/group/dpg_hgcal/comm_hgcal/hackathon/samples/close_by_double_pion/production/new_new_ntuples/"
files = glob.glob(f"{input_folder}/*ntuples_*.root")

calos = [ ]
tracksters = [ ]
associations = [ ]
graph = [ ]

i = 0
N = 10000000
for file in files[:3]:
    if i >= N: break
    i+=1
    try:
        print('.', end="")
        f = uproot.open(file)
        t =  f["ntuplizer/tracksters"]
        calo = f["ntuplizer/simtrackstersCP"]
        ass = f["ntuplizer/associations"]
        gra = f["ntuplizer/graph"]
        calos.append(calo.arrays(["stsCP_trackster_barycenter_eta","stsCP_trackster_barycenter_phi",
                                  "stsCP_barycenter_x","stsCP_barycenter_y","stsCP_barycenter_z","stsCP_raw_energy"]))
        tracksters.append(t.arrays(["NTracksters","raw_energy","raw_em_energy", "trackster_barycenter_eta","trackster_barycenter_phi",
                                    "barycenter_x","barycenter_y","barycenter_z","id_probabilities",
                                    "EV1", "EV2", "EV3", "eVector0_x", "eVector0_y","eVector0_z", "sigmaPCA1", "sigmaPCA2", "sigmaPCA3"]))
        associations.append(ass.arrays([ "tsCLUE3D_recoToSim_CP", "tsCLUE3D_recoToSim_CP_score"]))
        graph.append(gra.arrays(["linked_inners"]))
    except:
        print("error ", file)
        
df_calo = ak.concatenate(calos)
df_track = ak.concatenate(tracksters)
df_ass = ak.concatenate(associations)
df_gra = ak.concatenate(graph)

...

In [184]:
X = [ ]
Edges = [ ]
Edges_labels = [ ]

for ev in range(100):
#Get the tracksters in the window of each calo particles
# tracks_in_window = track_idx_inwindow[calo_idx_inwindow == calo_idx]
# Get only those trackers by ID
    trk_data = df_track[ev]
    gra_data = df_gra[ev]
    ass_data = df_ass[ev]
    # Save the input variables
    x_ev = ak.zip({"raw_en": trk_data.raw_energy, 
                'raw_em_energy': trk_data.raw_em_energy,
                     "barycenter_x": trk_data.barycenter_x,
                     "barycenter_y": trk_data.barycenter_y,
                     "barycenter_z": trk_data.barycenter_z,
                     "EV1": trk_data.EV1,
                     "EV2": trk_data.EV2,
                     "EV3": trk_data.EV3
                    }  )
    X.append(x_ev)

    nodes = []
    edges = []
    edges_labels = []
    for i in range(trk_data.NTracksters):
        nodes.append(i)
        qualities = ass_data.tsCLUE3D_recoToSim_CP_score[i]
        best_sts_i = ass_data.tsCLUE3D_recoToSim_CP[i][ak.argmin(qualities)]
        best_sts_i = best_sts_i if qualities[best_sts_i]<0.1 else -1
        for j in gra_data.linked_inners[i]:
            edges.append([j,i])
            qualities = ass_data.tsCLUE3D_recoToSim_CP_score[j]
            best_sts_j = ass_data.tsCLUE3D_recoToSim_CP[j][ak.argmin(qualities)]
            best_sts_j = best_sts_j if qualities[best_sts_j]<0.1 else -1
            if best_sts_i == best_sts_j:
                edges_labels.append(1)
            else:
                edges_labels.append(0)
            
    ed_np = np.array(edges).T
    Edges.append(ed_np)
    Edges_labels.append(edges_labels)


In [185]:
#print(Edges)

In [186]:
# Truth from TS->STS associations
ts_best_matches = [] # STS (from CP) best matched to a TS, can also be -1 for no match  
assoc_threshold = 0.1 # match only if < threshold
to_same = [] # TSs matched to the same STS grouped

        
for i in range(tracksterN[event]):
    qualities = tsAssocQual[event][i]
    best_score = np.amin(qualities) #lowest score
    assocmapnp = tsAssocMap[event][i]
    best_score_sts = assocmapnp[np.argmin(qualities)] #sts with the lowest score
    if best_score < assoc_threshold:
        ts_best_matches.append(best_score_sts)
    else:
        ts_best_matches.append(-1)
        
        
for sts in range(simTracksters_CP_N[event]+1):
    to_same.append([])
print("Best matches [sts]: ", ts_best_matches)
for ts, sts in enumerate(ts_best_matches):
    to_same[sts].append(ts)

# tracksters not truth-matched are put in the last element of to_same 
print("associated to_same:", to_same)


# TICLGraph #

edges = []
nodes = []
# pos in R-z, nx can't take 3D 
pos = {i:(z[event][i], np.sqrt(np.square(x[event][i]) + np.square(y[event][i]))) for i in range(tracksterN[event])}
for i in range(tracksterN[event]):
    nodes.append(i)
    for j in linkedInners[event][i]:
        edges.append([j,i])
print("number of TICLGraph edges: ", len(edges))
print(edges)
        

# TICLGraph truth #

# label TICLGraph edge 1 if the two edges are from the same CP else 0

edgesTICLtruth = []
for i in range(tracksterN[event]):
    for j in linkedInners[event][i]:
        if ts_best_matches[i] == ts_best_matches[j]:
            edgesTICLtruth.append(1)
        else:
            edgesTICLtruth.append(0)
print(len(edgesTICLtruth))

Best matches [sts]:  [1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1]
associated to_same: [[1, 4, 5, 13, 15, 16], [0, 2, 3, 6, 7, 8, 9, 10, 11, 12, 14, 17], []]
number of TICLGraph edges:  81
[[2, 0], [0, 3], [2, 3], [1, 4], [1, 5], [4, 5], [0, 6], [8, 6], [9, 6], [2, 6], [7, 6], [10, 6], [3, 6], [0, 7], [8, 7], [9, 7], [2, 7], [10, 7], [3, 7], [0, 8], [2, 8], [10, 8], [3, 8], [0, 9], [8, 9], [2, 9], [10, 9], [3, 9], [0, 10], [2, 10], [3, 10], [0, 11], [8, 11], [9, 11], [2, 11], [7, 11], [10, 11], [5, 12], [11, 12], [0, 12], [8, 12], [9, 12], [2, 12], [7, 12], [10, 12], [3, 12], [6, 12], [5, 13], [1, 13], [4, 13], [11, 14], [0, 14], [8, 14], [12, 14], [9, 14], [2, 14], [7, 14], [10, 14], [3, 14], [6, 14], [5, 15], [1, 15], [4, 15], [13, 15], [12, 15], [5, 16], [15, 16], [1, 16], [4, 16], [13, 16], [12, 16], [0, 17], [8, 17], [14, 17], [12, 17], [9, 17], [2, 17], [7, 17], [10, 17], [3, 17], [6, 17]]
81


In [187]:
edges_np = np.array(edges)

In [188]:
print(edges_np.T)

[[ 2  0  2  1  1  4  0  8  9  2  7 10  3  0  8  9  2 10  3  0  2 10  3  0
   8  2 10  3  0  2  3  0  8  9  2  7 10  5 11  0  8  9  2  7 10  3  6  5
   1  4 11  0  8 12  9  2  7 10  3  6  5  1  4 13 12  5 15  1  4 13 12  0
   8 14 12  9  2  7 10  3  6]
 [ 0  3  3  4  5  5  6  6  6  6  6  6  6  7  7  7  7  7  7  8  8  8  8  9
   9  9  9  9 10 10 10 11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12 13
  13 13 14 14 14 14 14 14 14 14 14 14 15 15 15 15 15 16 16 16 16 16 16 17
  17 17 17 17 17 17 17 17 17]]


In [189]:
# Model 
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 300
batch_size = 256

def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))

    return keras.Sequential(fnn_layers, name=name)

class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.2,
        aggregation_type="mean",
        combination_type="concat",
        normalize=False,
        *args,
        **kwargs,
    ):
        super(GraphConvLayer, self).__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        if self.combination_type == "gated":
            self.update_fn = layers.GRU(
                units=hidden_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                dropout=dropout_rate,
                return_state=True,
                recurrent_dropout=dropout_rate,
            )
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations, weights=None):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        if weights is not None:
            messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages):
        # node_indices shape is [num_edges].
        # neighbour_messages shape: [num_edges, representation_dim].
        num_nodes = tf.math.reduce_max(node_indices) + 1
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")

        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        # node_repesentations shape is [num_nodes, representation_dim].
        # aggregated_messages shape is [num_nodes, representation_dim].
        if self.combination_type == "gru":
            # Create a sequence of two elements for the GRU layer.
            h = tf.stack([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            # Concatenate the node_repesentations and aggregated_messages.
            h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            # Add node_repesentations and aggregated_messages.
            h = node_repesentations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        # Apply the processing function.
        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):
        """Process the inputs to produce the node_embeddings.

        inputs: a tuple of three elements: node_repesentations, edges, edge_weights.
        Returns: node_embeddings of shape [num_nodes, representation_dim].
        """

        node_repesentations, edges, edge_weights = inputs
        # Get node_indices (source) and neighbour_indices (target) from edges.
        node_indices, neighbour_indices = edges[0], edges[1]
        # neighbour_repesentations shape is [num_edges, representation_dim].
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)
        pr

        # Prepare the messages of the neighbours.
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
        # Aggregate the neighbour messages.
        aggregated_messages = self.aggregate(node_indices, neighbour_messages)
        # Update the node embedding with the neighbour messages.
        return self.update(node_repesentations, aggregated_messages)

class GNNEdgeClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        super(GNNEdgeClassifier, self).__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create the second GraphConv layer.
        self.conv2 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a layer which takes a pair of nodes connected by an edge, 
        # passes their embeddings through an ffn to produce the edge_embeddings
        self.edgespostprocess = create_ffn(hidden_units, dropout_rate, name="edgepostprocess")
        # Create a compute logits layer.
        self.compute_logits = layers.Dense(units=num_classes, name="logits")

    def call(self, input_node_indices):
        # Preprocess the node_features to produce node representations.
        x = self.preprocess(self.node_features)
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x1 + x
        # Apply the second graph conv layer.
        x2 = self.conv2((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x2 + x
        
        # Postprocess node embedding. <- being done in the edgepostprocess
        # x = self.postprocess(x)
        # Fetch node embeddings for the input node_indices.
        # node_embeddings = tf.gather(x, input_node_indices)
        
        # need a way to return the node_embeddings of the nodes
        # connected by the edges to be fed into an ffn that combines
        # them and feed these 'edge_embeddings' into a classification 
        # layer that predicts a label/score for every edge
        row = self.edges[0,:]
        col = self.edges[1,:]
        row_embed = tf.gather(x, row)
        col_embed = tf.gather(x, col)
        edge_embeddings = self.edgespostprocess(tf.concat([row_embed, col_embed],0))
        # Compute logits
        return self.compute_logits(edge_embeddings)


In [190]:
n_pad = 90
Node_data = []
Edge_data = [] # list of arrays !
for ev in range(10):
    X_ev = []
    Edge_data.append(Edges[ev])
    for fields in X[ev].fields:
        X_ev.append(np.pad(ak.to_numpy(X[ev][fields]), (0, 90-len(X[ev][fields]))))
    
    Node_data.append(X_ev)
Node_data = np.array(Node_data)

In [198]:
# Create an edges array (sparse adjacency matrix) of shape [2, num_edges].
edges = Edge_data[ev]
print(edges.shape[1])
# Create an edge weights array of ones.
edge_weights = tf.ones(shape=edges.shape[1])
# Create a node features array of shape [num_nodes, num_features].
node_features = tf.cast(
    Node_data[ev].T, dtype=tf.dtypes.float32
)
# Create graph info tuple with node_features, edges, and edge_weights.
graph_info = (node_features, edges, edge_weights)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)
    
gnn_model = GNNEdgeClassifier(
    graph_info=graph_info,
    num_classes=2,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

print("GNN output shape:", gnn_model(range(90)))

gnn_model.summary()

1461
Edges shape: (2, 1461)
Nodes shape: (90, 8)


InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [90,32] vs. shape[1] = [63,32] [Op:ConcatV2] name: concat